<div style="padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:pink;
           font-size:250%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow:1px 1px 10px 7px gray;
           letter-spacing:0.5px;
            margin: 15px 15px 15px 15px"><h1>Label Blur Detection usinG CNN</h1></div> 

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>📚 1. Imports</h2></div>

In [ ]:
# system library
import os

# math and tables
import pandas as pd
import numpy as np

# for model building
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# visualization libraries
import cv2
import matplotlib.pyplot as plt

# some utils
from sklearn.model_selection import train_test_split
from random import randint

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;
            margin: 15px 15px 15px 15px'><h2>📥 2. Data Loading, Formatting and Some Parameters</h2></div>

<div style="margin: 0px 0px 0px 0px;
            border-style: ridge;
            border-width: 0px;
            border-left-width: 5px;
            padding-left: 10px;">There are only 2 classes: <b>focus</b> and <b>blur</b>
File "metadata.csv" contains the file path location of each image and their labels. Images will be resized in process of loading. The size is stored in the variable <i>img_size</i>.</div>

In [ ]:
images_path = r'.\blur-detection-using-cnn\dataset\Images'

labels_df = pd.read_csv(r'.\blur-detection-using-cnn\dataset\metadata.csv')
print('\n\nlabels dataframe: \n', labels_df.head(), '\n\n')

class_names = ('blur','focus')
num_classes = len(class_names)

img_size = (128, 128, 3)

print(f'{num_classes} classes: {class_names}\nimage size: {img_size}')

labels = []
images = []
for image in labels_df.iloc:
    image_path = os.path.join(images_path, image[0])
    image_label = image[1]
    
    loaded_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    if loaded_image is not None and loaded_image.size != 0:
        resized_image = cv2.resize(loaded_image, img_size[0:2])[:, :, ::-1]
        images.append(np.asarray(resized_image))
        
        # labels will be in the form of a vector: [0, 1] or [1, 0]
        label = np.zeros(num_classes)
        label[image_label] = 1
        labels.append(label)
    else:
        print(f"Failed to load or empty image: {image_path}")

labels = np.asarray(labels)
images = np.asarray(images)

print(f'\nlabels shape: {labels.shape}')
print(f'images shape: {images.shape}')



<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>🏞️ 3. Data Visualisation</h2></div>

In [ ]:
# Display 16 pictures from the dataset
fig, axs = plt.subplots(4, 4, figsize=(10, 10))

for x in range(4):
    for y in range(4):
        i = randint(0, len(images) - 1)  # Generate a random index within the bounds

        axs[x][y].imshow(images[i])

        # Delete x and y ticks and set x label as picture label
        axs[x][y].set_xticks([])
        axs[x][y].set_yticks([])
        axs[x][y].set_xlabel(class_names[np.argmax(labels[i])])

plt.show()

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>🪓 4. Train/Validation Split</h2></div>

<div style="margin: 0px 0px 0px 0px;
            border-style: ridge;
            border-width: 0px;
            border-left-width: 5px;
            padding-left: 10px;">Data will be split into training and validation category.</div>

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=42)

print(f'train images shape: {X_train.shape}\ntrain labels shape: {y_train.shape}\n\nvalidation images shape: {X_val.shape}\nvalidation labels shape: {y_val.shape}\n')

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>➕ 5. Data Augmentation</h2></div>

<div style="margin: 0px 0px 0px 0px;
            border-style: ridge;
            border-width: 0px;
            border-left-width: 5px;
            padding-left: 10px;">The dataset is small, so use data augmentation.</div>

In [ ]:
# ImageDataGenerator for train images
train_images_generator = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.3,
                                                                         rotation_range=15,
                                                                         zoom_range=0.3,
                                                                         vertical_flip=True, 
                                                                         horizontal_flip=True)
train_images_generator = train_images_generator.flow(X_train, y=y_train)

# ImageDataGenerator for validation images
validation_images_generator = tf.keras.preprocessing.image.ImageDataGenerator(vertical_flip=True, 
                                                                              horizontal_flip=True)
validation_images_generator = validation_images_generator.flow(X_val, y=y_val)

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>6. CNN Model</h2></div>

<div style="margin: 0px 0px 0px 0px;
            border-style: ridge;
            border-width: 0px;
            border-left-width: 5px;
            padding-left: 10px;">The cnn_model will consist of an input <i>rescaling layer</i>, 3 <i>convolution layers blocks</i> and 4 <i>dense layers</i> after them. Input <i>rescaling layer</i>, which converts the values into a format from 0 to 1. The <i>block of convolution layers</i> contains 2 <i>convolution layers</i> and 1 <i>max pooling layer</i> after them (+ <i>global average pooling</i> if it is a jump from <i>convolutional layers</i> to <i>dense</i>). And the output <i>dense layer</i> contains as many neurons as there are classes.</div>

In [ ]:
def get_model():
    cnn_model = tf.keras.Sequential()

    # Inputs and rescaling
    cnn_model.add(tf.keras.layers.Rescaling(scale=1. / 255, input_shape=(img_size[0], img_size[1], img_size[2])))

    # Convolutional block 1
    cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    # Convolutional block 2
    cnn_model.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    # Convolutional block 3
    cnn_model.add(tf.keras.layers.Conv2D(256, (2, 2), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.Conv2D(256, (2, 2), activation='relu', padding='same'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    cnn_model.add(tf.keras.layers.GlobalAveragePooling2D())

    # Dense block
    cnn_model.add(tf.keras.layers.Dense(512, activation='relu'))
    cnn_model.add(tf.keras.layers.Dense(256, activation='relu'))
    cnn_model.add(tf.keras.layers.Dense(128, activation='relu'))
    cnn_model.add(tf.keras.layers.Dense(64, activation='relu'))
    cnn_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


    cnn_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return cnn_model

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>📈 7. Training and Results</h2></div>

In [ ]:
# creating ModelChecpoint callback
checkpoint_callback = ModelCheckpoint('cnn_model/model{epoch:02d}')

In [ ]:
cnn_model=get_model()
history = cnn_model.fit(train_images_generator, epochs=100, verbose=1, validation_data=validation_images_generator, callbacks=[checkpoint_callback])

<div style="margin: 0px 0px 0px 0px;
            border-style: ridge;
            border-width: 0px;
            border-left-width: 5px;
            padding-left: 10px;">Epoch is one entire passing of training dataset through the algorithm. Withe ach epoch we calculate the accuracy and loss and find the model in the best condition later. For example: load model with highest accuracy eg: model200</div> 

In [ ]:
# loading best model
cnn_model = tf.keras.models.load_model(r'.\blur-detection-using-cnn\cnn_model\model100')

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>📈 8. Accuracy and Loss Plot</h2></div>

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

In [ ]:
plt.figure()
plt.plot(epochs, accuracy, label='Training Accuracy')
plt.plot(epochs, loss, label='Training Loss')
plt.legend()
plt.title('Training Accuracy and Loss')

plt.figure()
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.title('Validation Accuracy and Loss')

plt.show()

<div style='padding: 4px;
            padding-left: 7px;
           display:fill;
           border-radius:20px;
           background-color:white;
           font-size:200%;
           font-family: Calibri;
           color:black;
           text-shadow: 1px 1px 2px black;
           box-shadow: 10px 7px 4.5px gray;
           letter-spacing:0.5px;margin: 15px 15px 15px 15px'><h2>9. Predictions</h2></div>

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(12, 12))

i = 0
for x in range(5):
    for y in range(4):
        prediction = cnn_model.predict(X_val[i][None, ...], verbose=0)[0]
        
        axs[x][y].set_xticks([])
        axs[x][y].set_yticks([])
        
        if np.argmax(prediction) != np.argmax(y_val[i]):   
            axs[x][y].set_xlabel(f'prediction: {class_names[np.argmax(prediction)]} | label: {class_names[np.argmax(y_val[i])]}', color='red')
        else:
            axs[x][y].set_xlabel(f'prediction: {class_names[np.argmax(prediction)]} | label: {class_names[np.argmax(y_val[i])]}')
        
        axs[x][y].imshow(X_val[i])
        
        i += 1
plt.show()